In [ ]:
from isatools.model import *

# Create a repeated treatment design with ISA descriptor

This example creates ISA descriptor for study with sequential treatments organized in an arm. This shows how to use objects from the isatools.create component in a granular fashion. It creates each element of the arm at a time.
Finally, the study design plan is shown by serializing the ISA Model content intermediate design_ISA-JSON.

## Study metadata

In [ ]:
from ipywidgets import RadioButtons, VBox, HBox, Layout, Label, Text, Textarea, DatePicker
import datetime
identifier = Text(
        value='S1',
        placeholder='e.g. S1',
        description='Identifier:',
        disabled=False
    )
title = Text(
        value='My Simple ISA Study',
        placeholder='e.g. My Study title',
        description='Title:',
        disabled=False
    )
description = Textarea(
    value="We could alternatively use the class constructor's parameters to set some default " \
          "values at the time of creation, however we want to demonstrate how to use the " \
          "object's instance variables to set values.",
    placeholder='Type something',
    description='Description:',
    disabled=False
)
sub_date = DatePicker(description="Submission date:", value=datetime.datetime.today())
release_date = DatePicker(description="Public release date:", value=datetime.datetime.today())
VBox([identifier,
      title,
      description,
      sub_date,
      release_date])

In [ ]:
investigation = Investigation()
study = Study(filename="s_study.txt")
study.identifier = identifier.value
study.title = title.value
study.description = description.value
study.submission_date = str(sub_date.value)
study.public_release_date = str(release_date.value)
study.sources = [Source(name="source1")]
study.samples = [Sample(name="sample1")]
study.protocols = [Protocol(name="sample collection")]
study.process_sequence = [Process(executes_protocol=study.protocols[-1], inputs=[study.sources[-1]], outputs=[study.samples[-1]])]
investigation.studies = [study]
investigation

In [ ]:
from isatools.isatab import dumps
print(dumps(investigation))

In [ ]:
import json
from isatools.isajson import ISAJSONEncoder
print(json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:
from isatools.create.models import * 


In [ ]:
# Creation of the first ISA element and setting its type
nte1 = NonTreatment(element_type='screen')
#print(nte1)

In [ ]:
te1 = Treatment()
#print(te1)

In [ ]:
te1.type='biological intervention'
#print(te1)

In [ ]:
#defining the first treatment as a vector of ISA factor values:
f1 = StudyFactor(name='virus')
f1v = FactorValue(factor_name=f1, value="hsv1")
f2 = StudyFactor(name='dose')
f2v = FactorValue(factor_name=f1, value='high dose')
f3 = StudyFactor(name='time post infection')
f3v = FactorValue(factor_name=f3, value='2', unit='hr')


In [ ]:
#assigning the factor values declared above to the ISA treatment element
te1.factor_values = [f1v,f2v,f3v]


In [ ]:
print(te1)

In [ ]:
# Creation of another ISA element, which is not a Treatment element
nte2 = NonTreatment()
# print(nte2)

In [ ]:
# setting its type by relying on the keys defined for the object
nte2.type=RUN_IN
#print(nte2)

In [ ]:
# changing the type of the ISA non treatment element using a string (note: the string should match exactly the predefined values)
nte2.type = 'follow-up'
#print(nte2)

In [ ]:
# setting the factor values associated with 'default' DURATION Factor associated with such elements
nte2.duration.value="2"
nte2.duration.unit="weeks"

In [ ]:
#print(nte2)

In [ ]:
# For each ISA element, creation of the associated container, known as an ISA Cell. In this example, each element is hosted by a single Cell
# However, for complex design, a cell may contain more than one element).
st_cl1= StudyCell(name="st_cl1", elements=[nte1])
st_cl2= StudyCell(name="st_cl2",elements=[te1])
st_cl3= StudyCell(name="st_cl3",elements=[nte2])


In [ ]:
# Creation of an ISA Study Arm and setting the number of subjects associated to that sequence of cells.
arm1 = StudyArm(name='study arm one as test')
arm1.group_size=20
print(arm1)

In [ ]:
# Declarationg of an ISA Sample Assay Plan, which defines which specimen are to be collected and which assays to be used
sap1 = SampleAssayPlan(name='sample_assay_plan1')


In [ ]:
# specifying which specimen
sap1.add_sample_type('liver')

In [ ]:
# specifying how many times each specimen is supposed to be collected
sap1.add_sample_plan_record('liver',3)

In [ ]:
# Declaration of an ISA assay and linking specimen type and data acquisition plan for this assay
assay_type1 = AssayType(measurement_type='metabolite profiling', technology_type='mass spectrometry')
sap1.add_assay_type(assay_type1)
sap1.add_assay_plan_record('liver',assay_type1)

In [ ]:
# Now starting to build an ISA Study Design Arm by adding the first set of ISA Cell and setting the Sample Assay Plan
arm1.add_item_to_arm_map(st_cl1,sap1)
print(arm1)



In [ ]:
# Now expanding the arm by adding a new cell, which uses the same Sample Assay Plan as the one used in Cell #1.
# Of course, the Sample Assay Plan for this new cell could be different. It would have to be to built as shown before.
arm1.add_item_to_arm_map(st_cl2,sap1)

In [ ]:
# Adding the last section of the Arm, with a cell which also uses the same sample assay plan.
arm1.add_item_to_arm_map(st_cl3,sap1)

In [ ]:
# Creating an ISA Study Design object, which will receive the arms defined by the user.
sd = StudyDesign(name='trial design #1')
print(sd)

In [ ]:
# Adding a study arm to the study design object.
sd.add_study_arm(arm1)
print(sd)

In [ ]:
# Let's now serialize the ISA study design to JSON
import json
from isatools.isajson import ISAJSONEncoder
from isatools.create.models import StudyDesignEncoder

print(json.dumps(sd, cls=StudyDesignEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:
# let's produce a graphical overview of the study design arms and the associated sample assay plans
TODO: